Using the latest cached version of the module from /home/Florian.Borchert/.cache/huggingface/modules/datasets_modules/datasets/distemist/0ae27654139a0655a3f3e3e4340b90cd5903357d2d8ae80b686927a45df57523 (last modified on Thu Apr 13 21:07:27 2023) since it couldn't be found locally at distemist., or remotely on the Hugging Face Hub.


Generating train split: 0 examples [00:00, ? examples/s]

Dataset distemist downloaded and prepared to /home/Florian.Borchert/.cache/huggingface/datasets/distemist/distemist_linking_bigbio_kb/1.0.0/0ae27654139a0655a3f3e3e4340b90cd5903357d2d8ae80b686927a45df57523. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 583
    })
})